### Writing the StockOption class

You can save the following code to a file named *StockOption.py*:

In [8]:
""" Store common attributes of a stock option """
import math
class StockOption(object):
    def __init__(self, S0, K, r, T, N, params): 
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N) # Ensure N have at least 1 time step 
        self.STs = None # Declare the stock prices tree
        """ Optional parameters used by derived classes """ 
        self.pu = params.get("pu", 0) # Probability of up state
        self.pd = params.get("pd", 0) # Probability of down state
        self.div = params.get("div", 0) # Dividend yield
        self.sigma = params.get("sigma", 0) # Volatility
        self.is_call = params.get("is_call", True) # Call or put
        self.is_european = params.get("is_eu", True) # Eu or Am
        """ Computed values """
        self.dt = T/float(N) # Single time step, in years
        self.df = math.exp(-(r-self.div) * self.dt) # Discount factor

### Writing the BinomialEuropeanOption class

Save this code to a file named *BinomialEuropeanOption.py*:

In [16]:
""" Price a European option by the binomial tree model """
#from StockOption import StockOption

import math
import numpy as np

class BinomialEuropeanOption(StockOption):
    
    def __setup_parameters__(self):
        """ Required calculations for the model """
        self.M = self.N + 1 # Number of terminal nodes of tree 
        self.u = 1 + self.pu # Expected value in the up state 
        self.d = 1 - self.pd # Expected value in the down state 
        self.qu = (math.exp((self.r-self.div)*self.dt) -self.d
                  ) / (self.u-self.d) 
        self.qd = 1-self.qu
        
    def _initialize_stock_price_tree_(self):
        # Initialize terminal price nodes to zeros 
        self.STs = np.zeros(self.M)
        # Calculate expected stock prices for each node 
        for i in range(self.M):
            self.STs[i] = self.S0*(self.u**(self.N-i))*(self.d**i)
            
    def _initialize_payoffs_tree_(self):
        # Get payoffs when the option expires at terminal nodes 
        payoffs = np.maximum(0, (self.STs-self.K) if self.is_call else(self.K-self.STs))
        return payoffs
    
    def _traverse_tree_(self, payoffs):
        # Starting from the time the option expires, traverse
        # backwards and calculate discounted payoffs at each node 
        for i in range(self.N):
            payoffs = (payoffs[:-1] * self.qu + payoffs[1:] * self.qd) * self.df
        return payoffs
    
    def __begin_tree_traversal__(self):
        payoffs = self._initialize_payoffs_tree_() 
        return self._traverse_tree_(payoffs)
    
    def price(self):
        """ The pricing implementation """ 
        self.__setup_parameters__() 
        self._initialize_stock_price_tree_() 
        payoffs = self.__begin_tree_traversal__()
        return payoffs[0]  # Option value converges to first node

In [17]:
#from StockOption import StockOption
#from BinomialEuropeanOption import BinomialEuropeanOption
eu_option = BinomialEuropeanOption(50, 50, 0.05, 0.5, 2,
                                   {"pu": 0.2, "pd": 0.2, "is_call": False})

In [20]:
print(eu_option.price())

4.82565175125599


### Pricing American options with the BinomialTreeOption class

Save the file as *BinomialAmericanOption.py* with the following code:

In [41]:
""" Price a European or American option by the binomial tree """
#from StockOption import StockOption
import math
import numpy as np
class BinomialTreeOption(StockOption):
    
    def _setup_parameters_(self):
        self.u = 1 + self.pu 
        # Expected value in the up state 
        self.d = 1 - self.pd # Expected value in the down state 
        self.qu = (math.exp((self.r-self.div)*self.dt) - self.d)/(self.u-self.d) 
        self.qd = 1-self.qu
        
    def _initialize_stock_price_tree_(self): 
        # Initialize a 2D tree at T=0 
        self.STs = [np.array([self.S0])]
        # Simulate the possible stock prices path 
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches*self.u,[prev_branches[-1]*self.d])) 
            self.STs.append(st) # Add nodes at each time step
        
    def _initialize_payoffs_tree_(self):
        # The payoffs when option expires return 
        np.maximum(0, (self.STs[self.N]-self.K
                      ) if self.is_call else (self.K-self.STs[self.N]))
        
    def __check_early_exercise__(self, payoffs, node): 
        early_ex_payoff = (self.STs[node] - self.K
                          ) if self.is_call else (self.K - self.STs[node])
        return np.maximum(payoffs, early_ex_payoff)
    
    def _traverse_tree_(self, payoffs): 
        for i in reversed(range(self.N)):
            # The payoffs from NOT exercising the option 
            payoffs = (payoffs[:-1] * self.qu + payoffs[1:] * self.qd) * self.df
            # Payoffs from exercising, for American options 
            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs, i)
        return payoffs
    
    def __begin_tree_traversal__(self):
        payoffs = self._initialize_payoffs_tree_() 
        return self._traverse_tree_(payoffs)
    
    def price(self):
        self._setup_parameters_() 
        self._initialize_stock_price_tree_() 
        payoffs = self.__begin_tree_traversal__()
        return payoffs[0]

In [42]:
#from BinomialAmericanOption import BinomialTreeOption
am_option = BinomialTreeOption(50, 50, 0.05, 0.5, 2,
                            {"pu": 0.2, "pd": 0.2, "is_call": False, "is_eu": False})
print(am_option.price())

TypeError: 'NoneType' object is not subscriptable

### The Cox-Ross-Rubinstein model

$$u=e^{σ(∆t)^{1/2}}$$
$$d=frac{1}{U}=e^{−σ(∆tu)^{1/2}}$$

In [48]:
""" Price an option by the binomial CRR model """
#from BinomialTreeOption import BinomialTreeOption
import math
class BinomialCRROption(BinomialTreeOption):
    def _setup_parameters_(self):
        self.u = math.exp(self.sigma * math.sqrt(self.dt)) 
        self.d = 1./self.u
        self.qu = (math.exp((self.r-self.div)*self.dt) -self.d)/(self.u-self.d) 
        self.qd = 1-self.qu

In [49]:
#from BinomialCRROption import BinomialTreeOption

eu_option = BinomialCRROption(50, 50, 0.05, 0.5, 2,{"sigma": 0.3, "is_call": False})
print ("European put: %s" % eu_option.price())

am_option = BinomialCRROption(50, 50, 0.05, 0.5, 2,{"sigma": 0.3, "is_call": False, "is_eu": False})
print ("American put: %s" % am_option.price())

TypeError: 'NoneType' object is not subscriptable

### Using a Leisen-Reimer tree

In [55]:
""" Price an option by the Leisen-Reimer tree """ 
#from BinomialTreeOption import BinomialTreeOption 
import math
class BinomialLROption(BinomialTreeOption):
    def _setup_parameters_(self):
        odd_N = self.N if (self.N%2 == 1) else (self.N+1) 
        d1 = (math.log(self.S0/self.K) +
              ((self.r-self.div) + (self.sigma**2)/2.) *
              self.T) / (self.sigma * math.sqrt(self.T)) 
        d2 = (math.log(self.S0/self.K) +
              ((self.r-self.div) - (self.sigma**2)/2.) *
              self.T) / (self.sigma * math.sqrt(self.T)) 
        pp_2_inversion = lambda z, n: .5 + math.copysign(1, z) * math.sqrt(
            .25 - .25 * math.exp( -((z/(n+1./3.+.1/(n+1)))**2.)*(n+1./6.)
                                )) 
        pbar = pp_2_inversion(d1, odd_N)
        self.p = pp_2_inversion(d2, odd_N)
        self.u = 1/self.df * pbar/self.p
        self.d = (1/self.df - self.p*self.u)/(1-self.p) 
        self.qu = self.p
        self.qd = 1-self.p

In [57]:
#from BinomialLROption import BinomialLROption

eu_option = BinomialLROption(50, 50, 0.05, 0.5, 3,{"sigma": 0.3, "is_call": False})
print ("European put: %s" % eu_option.price())

am_option = BinomialLROption(50, 50, 0.05, 0.5, 3,{"sigma": 0.3, "is_call": False, "is_eu": False})
print ("American put: %s" % am_option.price())

TypeError: 'NoneType' object is not subscriptable

### The Greeks for free

In [63]:
""" Compute option price, delta and gamma by the LR tree """ 
#from BinomialLROption import BinomialLROption
import numpy as np
class BinomialLRWithGreeks(BinomialLROption):
    def __new_stock_price_tree__(self):
        """
        Create additional layer of nodes to our original stock price tree
        """
        self.STs = [np.array([self.S0*self.u/self.d, self.S0, self.S0*self.d/self.u])]
    for i in range(self.N):
        prev_branches = self.STs[-1]
        st = np.concatenate((prev_branches * self.u,[prev_branches[-1] * self.d]))
        self.STs.append(st)
    def price(self): 
        self._setup_parameters_() 
        self.__new_stock_price_tree__()
        payoffs = self.__begin_tree_traversal__()
        """ Option value is now in the middle node at t=0 """ 
        option_value = payoffs[len(payoffs)/2]
        payoff_up = payoffs[0] 
        payoff_down = payoffs[-1] 
        S_up = self.STs[0][0] 
        S_down = self.STs[0][-1] 
        dS_up = S_up - self.S0 
        dS_down = self.S0 - S_down
        """ Get delta value """
        dS = S_up - S_down
        dV = payoff_up - payoff_down 
        delta = dV/dS
        """ Get gamma value """
        gamma = ((payoff_up-option_value)/dS_up -
                 (option_value-payoff_down)/dS_down) / ((self.S0+S_up)/2. - (self.S0+S_down)/2.)
        return option_value, delta, gamma

NameError: name 'self' is not defined

In [65]:
#from BinomialLRWithGreeks import BinomialLRWithGreeks
eu_call = BinomialLRWithGreeks(50, 50, 0.05, 0.5, 300, {"sigma": 0.3, "is_call": True})
results = eu_call.price()
print ("European call values")
print ("Price: %s\nDelta: %s\nGamma: %s" % results)

eu_put = BinomialLRWithGreeks(50, 50, 0.05, 0.5, 300, {"sigma":0.3, "is_call": False})
results = eu_put.price()
print ("European put values")
print "Price: %s\nDelta: %s\nGamma: %s" % results)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Price: %s\nDelta: %s\nGamma: %s" % results))? (<ipython-input-65-1e2abcaba792>, line 10)

# *etc..  pg:90*